In [1]:
import os
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from folium.plugins import FastMarkerCluster
# Size matters
# with 1k circles html is 1MB
# To find other tilesets maybe accessible see https://leaflet-extras.github.io/leaflet-providers/preview/


def find_middle(input_list):
    middle = float(len(input_list))/2
    if middle % 2 != 0:
        return input_list[int(middle - .5)]
    else:

        return ( input_list[int(middle)]+ input_list[int(middle-1)])/2

C:\Users\pablo\AppData\Local\Temp\ipykernel_14240\3394799362.py:5: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
DIR_OUTPUT = "./callejeros"

# Prepare clusters, one per municipality
MARKER_CLUSTERS = []
CIRCLES = []

# For every COMUNIDAD AUTONOMA...
_comunidad = "comunidad-de-madrid"
_filepath = os.path.join(DIR_OUTPUT, _comunidad)

# ... load data
df = pd.read_excel(f"{_filepath}_streets.xlsx")
dispdf = df[df["Display"] == 1]
municipios = dispdf["Municipality"].unique()

# For every MUNICIPALITY...
for municipio in tqdm(municipios):
    # ... create marker cluster
    # MARKER_CLUSTERS.append(FastMarkerCluster(data=(),name=municipio))

    # ... load municipality data
    _gjson_fp = os.path.join(_filepath, f"{_comunidad}_{municipio}.geojson")
    _gjson = gpd.read_file(_gjson_fp)

    # ... list streets to be displayed on the map
    _mun_df = dispdf[dispdf["Municipality"] == municipio]
    _streets = _mun_df["Street_name"].to_list()

    # For every STREET to de displayed...
    for street in _streets:
        geometry = _gjson[_gjson["Name"] == street]["geometry"].iloc[0]
        street_x, street_y = geometry.coords.xy
        _x = find_middle(np.array(street_x))
        _y = find_middle(np.array(street_y))
        _mun = municipio.replace('-',' ').capitalize()
        popup = f"<b>{street}, {_mun}</b>"
        # folium.Marker([_y, _x], popup=popup).add_to(MARKER_CLUSTERS[-1])
        circ = folium.Circle(radius=50, 
                             location=(_y,_x),
                             popup=popup,
                             color="crimson",
                             fill=True )
        CIRCLES.append(circ)



100%|██████████| 102/102 [00:14<00:00,  7.18it/s]


In [16]:
m = folium.Map(location=[40.419302, -3.692759],
               zoom_start=6,
               tiles="OpenStreetMap",  #cartodbdarkmatter is also nice
               name ="OpenStreetMap",
               control_scale=True
              )
folium.TileLayer(tiles="cartodbpositron", attr='CartoDB', name="CartoDB light").add_to(m)
folium.TileLayer(tiles="cartodbdark_matter", attr="CartoDB", name="CartoDB dark").add_to(m)
# Create marker cluster, then add markers to cluster rather than to the map
# marker_cluster = folium.MarkerCluster("Cluster Name").add_to(my_map)
# folium.Marker([lon, lat]).add_to(marker_cluster)
#for mcluster in MARKER_CLUSTERS:
#    mcluster.add_to(m)

for circ in CIRCLES:
    circ.add_to(m)

# Add geoJson object
# if added from URL the map.html will weigh like a normal map
# if added from a local file, the map.html contains a copy of the geojson object
geojson_url = "https://raw.githubusercontent.com/pyubero/HPStvcIOWfCj/main/mappable.geojson"
folium.GeoJson(geojson_url, name='Lugares con "Mayor"').add_to(m)    

# Add list of coordinates as PolyLines
# folium.PolyLine(trail_coordinates, tooltip="Coast").add_to(m)


folium.LayerControl().add_to(m)
m.save("map.html")
